In [3]:
import pandas as pd
import os 
from utils import (
    normalizar_pais,
    agrupar_pais,
    comprobar_normalidad,
    correlacion_variables,
    comprobar_nomalidad_y_varianza,
    calcular_fuerza_asociacion_2_grupos,
    calcular_fuerza_asociacion_mas_2_grupos,
    cramers_v,
    calcular_independencia_categoricas,
    agrupar_tramo,
    cohens_d,
    r_mannwhitney,
    prueba_comparacion_2grupos_variable
)
